In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_ssd_model(num_classes):
    base_model = tf.keras.applications.MobileNetV2(input_shape=(640, 640, 3), include_top=False)
    base_out = base_model.output
    x = layers.Conv2D(4 * (num_classes + 4), kernel_size=(3, 3), padding='same')(base_out)
    model = models.Model(inputs=base_model.input, outputs=x)
    return model

model = create_ssd_model(num_classes=26)  # Change num_classes according to your dataset


2024-04-01 14:50:04.301540: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-01 14:50:04.320800: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-01 14:50:04.422077: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 14:50:04.422128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 14:50:04.441639: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

9406464/9406464 [==============================] - 1s 0us/step


In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

# Define your model architecture
def create_ssd_model(input_shape, num_classes):
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False)
    base_out = base_model.output
    x = layers.Conv2D(4 * (num_classes + 4), kernel_size=(3, 3), padding='same')(base_out)
    model = models.Model(inputs=base_model.input, outputs=x)
    return model

# Define your data loading pipeline
import json

def parse_coco_annotations(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    annotations = {}
    for image_data in data['images']:
        image_id = image_data['id']
        file_name = image_data['file_name']
        annotations[image_id] = {'file_name': file_name, 'objects': []}

    for annotation_data in data['annotations']:
        image_id = annotation_data['image_id']
        category_id = annotation_data['category_id']
        bbox = annotation_data['bbox']
        xmin, ymin, width, height = bbox
        xmax = xmin + width
        ymax = ymin + height
        annotations[image_id]['objects'].append({
            'category_id': category_id,
            'bbox': [xmin, ymin, xmax, ymax]
        })
    return annotations

# Example usage
coco_annotations = parse_coco_annotations('face cephalometry.v6i.coco/train/_annotations.coco.json')

# Set up your dataset
train_dataset = "face cephalometry.v6i.coco/train"  # Define your TensorFlow dataset for training
val_dataset = "face cephalometry.v6i.coco/valid"    # Define your TensorFlow dataset for validation

# Define input shape and number of classes
input_shape = (640, 640, 3)  # Adjust according to your image size
num_classes = 26  # Adjust according to the number of classes in your dataset

# Create and compile the model
model = create_ssd_model(input_shape, num_classes)
model.compile(optimizer=Adam(), loss=MeanSquaredError())

# Train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=10)

# Save the trained model
model.save("multiple_class_object_detection_model.h5")


2024-04-02 14:36:02.670208: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-02 14:36:02.692052: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-02 14:36:02.812725: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 14:36:02.812796: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 14:36:02.833286: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

2024-04-02 14:36:06.779338: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-02 14:36:06.794914: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


IndexError: tuple index out of range

In [10]:
import json

def parse_coco_annotations(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    annotations = {}
    for image_data in data['images']:
        image_id = image_data['id']
        file_name = image_data['file_name']
        annotations[image_id] = {'file_name': file_name, 'objects': []}

    for annotation_data in data['annotations']:
        image_id = annotation_data['image_id']
        category_id = annotation_data['category_id']
        bbox = annotation_data['bbox']
        xmin, ymin, width, height = bbox
        xmax = xmin + width
        ymax = ymin + height
        annotations[image_id]['objects'].append({
            'category_id': category_id,
            'bbox': [xmin, ymin, xmax, ymax]
        })
    return annotations

# Example usage
coco_annotations = parse_coco_annotations('face cephalometry.v6i.coco/train/_annotations.coco.json')


[491, 227, 7.5, 5.5]
[485, 247, 8, 5]
[421, 247, 7, 4.5]
[454, 236, 14, 3.5]
[450, 241, 18.5, 4.5]
[447, 258, 14.5, 3.5]
[454, 248, 6.5, 4.5]
[532, 309, 8, 5]
[502, 311, 7.5, 4.5]
[475, 317, 7, 5]
[469, 329, 7, 4.5]
[489, 329, 7.5, 4.5]
[489, 352, 7, 4.5]
[439, 368, 7, 5]
[472, 375, 7, 4.5]
[470, 402, 7.5, 5]
[443, 424, 7.5, 4.5]
[195, 237, 9.5, 6.5]
[161, 227, 8.5, 5.5]
[210, 324, 7.5, 5]
[395, 223, 8, 5.5]
[509, 324, 7.5, 4.5]
[139, 239, 7.5, 4.5]
[178, 249, 5, 4]
[150, 149, 8, 4.5]
[156, 243, 9, 3]
[160, 255, 16.5, 2.5]
[162, 248, 5.5, 3.5]
[110, 281, 6.5, 3.5]
[123, 284, 6.5, 3.5]
[134, 315, 6.5, 3.5]
[142, 290, 5.5, 3.5]
[149, 336, 6.5, 3.5]
[164, 330, 7.5, 5]
[163, 359, 7, 4.5]
[139, 298, 5.5, 3.5]
[184, 369, 6.5, 4.5]
[334, 254, 7.5, 5]
[153, 234, 15, 2.5]
[152, 300, 4.5, 3]
[358, 241, 7.5, 4.5]
[327, 324, 7.5, 5]
[125, 294, 6, 3.5]
[136, 222, 7.5, 5]
[185, 226, 7.5, 5]
[170, 264, 6, 4]
[159, 331, 6, 4]
[161, 350, 5, 4]
[198, 355, 6, 4]
[146, 388, 5.5, 4]
[173, 388, 6, 4]
[168, 